In [1]:
from src.learner import *
import time

from utilities import remove_cols


2024-02-18 08:43:06.831838: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-18 08:43:06.860513: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX_VNNI, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# inputs and outputs
X = np.genfromtxt('data/orth.csv', delimiter=",")
Y = np.genfromtxt('data/phon.csv', delimiter=",")

For tuning we will use a random sample of the same size that our samples will eventually be. This involves allocating 600 words for test, and the rest for train - but not using our pre-allocated samples for the purpose.

In [6]:
np.random.seed(982)

target_train_size = 300

train_n = X.shape[0]
test_n = train_n-target_train_size

sample = np.full(train_n, True, dtype=bool)

indices = np.random.choice(train_n, test_n, replace=False)

# Set chosen indices to True because they select the test items not the train items
sample[indices] = False

Limited search across HPs...

In [11]:
seed = 387

with open('outputs/tune_3k_1a.csv', 'w') as f:
    f.write("{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                            "hidden_units",
                                            "learning_rate",
                                             "batch_size",
                                             "epochs",
                                             "loss_train",
                                             "accuracy_train",
                                             "mse_train",
                                             "loss_test",
                                             "accuracy_test",
                                             "mse_test",
                                             "time"))
    for learning_rate in [.01, .025, .05, .075, .1, .15, .2, .25, None]: 
        for batch_size in [16, 32, 64, 96, 128, 256]:
            for epochs in [50, 100, 150, 200, 250, 300]:
                for hidden in [80, 100, 120]:

                    if learning_rate is not None:
                        optimizer = Adam(learning_rate=learning_rate)
                    if learning_rate is None:
                        optimzer = None

                    model = learner(X, Y, seed, hidden, optimizer=None)
                    
                    start_time = time.time()


                    model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=False)

                    end_time = time.time()
                    runtime = end_time - start_time

                    loss_train, accuracy_train, mse_train = model.evaluate(X[sample], Y[sample], verbose=0) 
                    loss_test, accuracy_test, mse_test = model.evaluate(X[~sample], Y[~sample], verbose=0) 

                    f.write("{},{},{},{},{},{},{},{},{},{},{}\n".format(
                                                    hidden,
                                                    learning_rate,
                                                    batch_size,
                                                    epochs,
                                                    loss_train,
                                                    accuracy_train,
                                                    mse_train,
                                                    loss_test,
                                                    accuracy_test,
                                                    mse_test,
                                                    runtime))
f.close()

Let's check out item-level performance on the sample for the following configuration:

hidden = 100
learning_rate = 0.01
batch_size = 16
epochs = 250
train_accuracy = 0.997
test_accuracy = 0.990
time = 4.73 seconds

In [17]:
hidden = 100
learning_rate = .01
batch_size = 16
epochs = 500

model = learner(X, Y, seed, hidden, optimizer=Adam(learning_rate=learning_rate))                    
model.fit(X[sample], Y[sample], epochs=epochs, batch_size=batch_size, verbose=True)

Epoch 1/500
19/19 [==============================] - 0s 1ms/step - loss: 0.2890 - binary_accuracy: 0.9168 - mse: 0.0884
Epoch 2/500
19/19 [==============================] - 0s 1ms/step - loss: 0.0847 - binary_accuracy: 0.9670 - mse: 0.0249
Epoch 3/500
19/19 [==============================] - 0s 1ms/step - loss: 0.0709 - binary_accuracy: 0.9713 - mse: 0.0211
Epoch 4/500
19/19 [==============================] - 0s 940us/step - loss: 0.0573 - binary_accuracy: 0.9778 - mse: 0.0167
Epoch 5/500
19/19 [==============================] - 0s 1ms/step - loss: 0.0463 - binary_accuracy: 0.9825 - mse: 0.0133
Epoch 6/500
19/19 [==============================] - 0s 1ms/step - loss: 0.0379 - binary_accuracy: 0.9860 - mse: 0.0106
Epoch 7/500
19/19 [==============================] - 0s 1ms/step - loss: 0.0311 - binary_accuracy: 0.9891 - mse: 0.0086
Epoch 8/500
19/19 [==============================] - 0s 1ms/step - loss: 0.0263 - binary_accuracy: 0.9910 - mse: 0.0070
Epoch 9/500
19/19 [===================

In [21]:
accuracy = batch_accuracy(Y, model.predict(X), dichotomous=False)

91/91 [==============================] - 0s 637us/step


In [20]:
sum([1 for acc in accuracy if acc == True])

616

In [31]:
preds = (model.predict(X) > .5).astype(int)

91/91 [==============================] - 0s 649us/step


In [38]:
np.savetxt('outputs/tune_3k_1a_prediction_accuracies_sample_run.csv', (preds == Y).astype(int), delimiter = ',')

Now let's take this configuration and run over a set of random seeds and generate some longitudinal data to examine. 